**bold text**##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Demo TF-Hub for video generation using Inbetweening 3d Convolutions

Yunpeng Li, Dominik Roblek, and Marco Tagliasacchi. From Here to There: Video Inbetweening Using Direct 3D Convolutions, 2019.

https://arxiv.org/abs/1905.10240


Current Hub characteristics:
- has models for BAIR Robot pushing videos and KTH action video dataset (though this colab uses only BAIR)
- BAIR dataset already available in Hub. However, KTH videos need to be supplied by the users themselves.
- only evaluation (video generation) for now
- batch size and frame size are hard-coded
- Tensorflow 1.x only

IMPORTANT NOTE: If an error of "Not enough disk space" appears, it's likely due to the downloading of the full dataset (~30GB). In that case, we recommend either connecting via a local runtime or downloading the dataset to Google Drive and loading it manually instead of calling tfds.load().

In [0]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
import tensorflow_datasets as tfds

## BAIR: Demo based on numpy array inputs

In [0]:
# @title Load some example data (BAIR).
batch_size = 16
with tf.Graph().as_default():
  # If unable to download the dataset automatically due to "not enough disk space", please download manually to Google Drive and
  # load using tf.data.TFRecordDataset.
  ds = tfds.load('bair_robot_pushing_small', split='test')
  test_videos = ds.batch(batch_size).make_one_shot_iterator().get_next()['image_aux1'][:, ::15]
  with tf.train.MonitoredSession() as sess:
    input_frames = sess.run(test_videos)

In [0]:
# @title Visualize loaded videos start and end frames.
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

print('Test videos shape [batch_size, start/end frame, height, width, num_channels]: ', input_frames.shape)
sns.set_style('white')
plt.figure(figsize=(4, 2*batch_size))

for i in range(batch_size)[:4]:
  plt.subplot(batch_size, 2, 1 + 2*i)
  plt.imshow(input_frames[i, 0])
  plt.title('Video {}: First frame'.format(i))
  plt.axis('off')
  plt.subplot(batch_size, 2, 2 + 2*i)
  plt.imshow(input_frames[i, 1])
  plt.title('Video {}: Last frame'.format(i))
  plt.axis('off')

### Load Hub Module

In [0]:
tf.reset_default_graph()

In [0]:
hub_handle = 'https://tfhub.dev/google/tweening_conv3d_bair/1'
module = hub.Module(hub_handle)

### Generate and show the videos

In [0]:
# Expected inputs as a placeholder.
inputs_ph = tf.placeholder(dtype=tf.float32, shape=(None, 2, None, None, 3))
outputs = module(inputs_ph)

# Run graph for given numpy arrays.
with tf.train.MonitoredSession() as sess:
  filled_frames = sess.run(outputs, feed_dict={inputs_ph: input_frames}).astype(np.uint8)

In [0]:
# Show sequences of generated video frames.

# Concatenate start/end frames and the generated filled frames for the new videos.
generated_videos = np.concatenate([input_frames[:, :1], filled_frames, input_frames[:, 1:]], axis=1)

for i in range(4):
  fig = plt.figure(figsize=(10*2, 2))
  for j in range(1, 16):
    ax = fig.add_axes([j*1/16., 0, (j+1)*1/16., 1], xmargin=0, ymargin=0)
    ax.imshow(generated_videos[i, j])
    ax.axis('off')